<a href="https://colab.research.google.com/github/purang2/BERT/blob/main/AEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets #from huggingface
!pip install evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install koeda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Testing

In [3]:
from koeda import AEDA


aeda = AEDA(
    morpheme_analyzer="Okt", punc_ratio=0.3, punctuations=[".", ",", "!", "?", ";", ":"]
)

text = "어머니가 집을 나가신다"

result = aeda(text)
print(result)
# 어머니가 ! 집을 , 나가신다

result = aeda(text, p=0.9, repetition=2)
print(result)


; 어머니 ! 가 집을 나가신다
['어머니가 : 집 ? 을 : 나가신다', '어머니가 : 집 . 을 나가신다']


## Main: Run for our Data

In [4]:
from datasets import load_dataset
import pandas as pd 

data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"


train_data = pd.read_csv(data_path+"/train_data.tsv", sep='\t')
train_labels = pd.read_csv(data_path+"/train_labels.csv")

train_data

,sentence_id,person,sentence,scene,context
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는"
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN


In [5]:
train_drop = train_data[['person', 'sentence', 'context']]
train_drop['label'] = train_labels['label']
train_pair = train_drop

import numpy as np 

train_pair['context2'] = train_pair['context'].fillna('none')
train_pair['sentence2'] = train_pair['sentence'].fillna('none')

train_pair


train_pair = train_pair[['person', 'context2', 'sentence2', 'label']]
train_pair['scene'] = train_data['scene']
train_pair

train = list()
scene_df = train_pair['scene']
scene_df = scene_df.drop_duplicates().reset_index(drop=True)


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario
  elif(0<i and i<len(df)-1 ):
    scenario = scenario + df.iloc[i-1]['person'] + "가 말하길 " +"\""+ df.iloc[i-1]['sentence2']+"\""+ " " + df.iloc[i]['person'] + "가 말하길 " +"\""+df.iloc[i]['sentence2'] +"\""+" "+df.iloc[i+1]['person'] + "가 말하길 " +"\""+ df.iloc[i+1]['sentence2']+"\""
    return scenario
  elif(i == len(df)-1):
    scenario = scenario + df.iloc[i-1]['person'] + "가 말하길 " +"\""+ df.iloc[i-1]['sentence2'] +"\""+" "+ df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario


def trans_df(train_pair, train):
  for i in scene_df[:]:
    temp = train_pair[train_pair['scene']==i].reset_index(drop=True)
    train+=[mk_scenario(j, temp) for j in range(len(temp))]
    
trans_df(train_pair, train)
train = pd.DataFrame(train, columns=('scenario',))
print(train.iloc[1]['scenario'])

train['label'] = train_pair['label']
train
train_pairs = train

labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

train_pairs['label'] = [labels[label] for label in train_pairs['label']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep> 어영가 말하길 "우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다."


once you use it, then off

In [6]:
#train_pairs.to_csv(data_path+'/train_pairs.csv', index=False)
#train_pairs


In [7]:
train_pairs = pd.read_csv(data_path+"/train_pairs.csv", encoding='utf-8')
train_pairs

,scenario,label
0,"none <sep> 어영가 말하길 ""야! 전화 받아. 아무리 바빠도 내전화는 받아야...",2
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep>...",2
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,1
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,2
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,0
...,...,...
7334,"none <sep> 건강가 말하길 ""행복할 일도 많다. 이까짓거 갖고. 일단 끊어라...",0
7335,"none <sep> 청난가 말하길 ""종남이 맛있는것도 사주고 잘 챙겨줘."" 건강가 ...",0
7336,"none <sep> 건강가 말하길 ""걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다...",0
7337,"none <sep> 종남가 말하길 ""아빠. 우리 저거 타."" 건강가 말하길 ""나 못...",0


In [8]:
pd.DataFrame(list(train_pairs['scenario']))

,0
0,"none <sep> 어영가 말하길 ""야! 전화 받아. 아무리 바빠도 내전화는 받아야..."
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep>..."
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...
...,...
7334,"none <sep> 건강가 말하길 ""행복할 일도 많다. 이까짓거 갖고. 일단 끊어라..."
7335,"none <sep> 청난가 말하길 ""종남이 맛있는것도 사주고 잘 챙겨줘."" 건강가 ..."
7336,"none <sep> 건강가 말하길 ""걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다..."
7337,"none <sep> 종남가 말하길 ""아빠. 우리 저거 타."" 건강가 말하길 ""나 못..."


In [56]:
len(list(train_pairs['scenario'])[7300:7400])

39

In [53]:
73/3

24.333333333333332

In [62]:
results = []

j_list = []

In [115]:
aeda = AEDA(
    morpheme_analyzer="Okt", punc_ratio=0.3, punctuations=[".", ",", "!", "?", ";", ":"]
)

text = "어머니가 집을 나가신다"
text = list(train_pairs['scenario'])


#result = aeda(text)
#print(result)
# 어머니가 ! 집을 , 나가신다

#result = aeda(text, p=0.5, repetition=2)

#results = []


'''
for i in range(15):
  j_list.append(i)
'''
print(j_list) 
for i in range(74):  
  for j in range(2*i,(2*i)+2):    
    if j in j_list: 
      continue
    result = aeda(text[j*50:(j*50)+50])
    results.extend(result)
    print(j)
    j_list.append(j)
      

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143]
144
145
146
147


In [116]:
j_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147]

In [117]:
len(results)

7339

In [123]:
results

['none <sep> ; 어영가 ? 말하길 "야! 전화 받아 : . 아무리 바빠도 내전화는 받아야 : 되는거 아냐 , ? 약속 , 하나도 . 못지키는 ! 주제에 법 ! 을 ; 지켜?"',
 '포기한듯 탁 일어서는데, 띵동 문자 , . 후다닥 보는 어영. 기막혀 읽어보는 , <sep> ! 어영가 ; 말 ; 하길 " ! 우리 . 아무래도 안되겠다. 이게 최선인거 같애. 평생 : 잊지 않을게. ; 행복하길 바란다."',
 '초조하게 시계보면서 왔다 ? 갔다 : 서성이는 김순경 . . 순찰차(경차) . 와서 ? 멈추고 내리는 지구대 <sep . > 지구대 : 가 말하길 ? " , 김경사님 ; , 아직 : 안가셨어요? 시간 다 ; 됐을텐데."',
 '; 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차 ! (경차)와서 멈추고 내리는 지구대 <sep> ; 지구대가 . 말하길 "김경사님, 아직 안가셨어요? 시간 다됐을텐데." 순경가 말하길 "근무중인데 어딜가?" 지구대가 말 . 하길 "다녀오세요. 이런 , 날은 무조건 가서 축하해주셔야죠. : 이순경이 대신 근무선다고 나온대요."',
 ', 초조하게 시계 , 보면서 왔다 ? 갔다 서성이는 김순경 ? . 순찰차( . 경차)와서 멈추고 내리는 : 지구대 <sep> 순경가 말하길 "근무중인데 어딜 ! 가?" . 지구대가 말하길 : "다녀오세요. , 이런날 ; 은 : 무조건 가서 축하해주셔야죠 ; . ! 이순경이 ? 대신 ? 근무선다고 나온대요." 순경가 ? 말하길 "그럴거 . 없어."',
 '초조하게 시계보면서 왔다 , 갔다 서성이는 김순경. . 순찰차(경차)와서 멈추고 : 내리는 지구대 : < ! sep> 지구대가 말하길 "다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요." 순경가 말하길 "그럴거 없어." 지구대가 말하길 "막내 ? 아드님 모범경찰상 받으시는데 직접 보셔야죠. 얼마나 자랑스럽습니까?"',
 '초조하게 ? 시계보면서 왔다갔다 서성이는 ; 김순경. 순찰차(경차)와서 멈추고 

In [133]:

results_data = pd.DataFrame(results)
results_data['label'] = train_labels['label']



labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

results_data['label'] = [labels[label] for label in results_data['label']]

results_data.columns=['scenario','label']
#pd.DataFrame(results).to_csv(data_path+'/aeda_7338.csv',index=False)
results_data

,scenario,label
0,"none <sep> ; 어영가 ? 말하길 ""야! 전화 받아 : . 아무리 바빠도 내...",2
1,"포기한듯 탁 일어서는데, 띵동 문자 , . 후다닥 보는 어영. 기막혀 읽어보는 , ...",2
2,초조하게 시계보면서 왔다 ? 갔다 : 서성이는 김순경 . . 순찰차(경차) . 와서...,1
3,; 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차 ! (경차)와서 멈추고 내...,2
4,", 초조하게 시계 , 보면서 왔다 ? 갔다 서성이는 김순경 ? . 순찰차( . 경차...",0
...,...,...
7334,"none , < : sep> 건강가 말하길 ""행복할 ; 일도 많다. 이까짓 ! 거 ...",0
7335,"none < : sep> ? 청난가 말하길 ""종남 ? 이 맛있는것도 사주고 ? 잘 ...",0
7336,"none <sep> 건강가 말 ? 하길 . ""걱정마라. 먹어도 : 같이 먹고 굶어도...",0
7337,", none <sep> 종남 : 가 ? 말하길 ""아빠. 우리 저거 타."" ! 건강가...",0


In [134]:
ss = pd.read_csv(data_path+'/aeda_7338.csv')

ss

,0
0,"none <sep> ; 어영가 ? 말하길 ""야! 전화 받아 : . 아무리 바빠도 내..."
1,"포기한듯 탁 일어서는데, 띵동 문자 , . 후다닥 보는 어영. 기막혀 읽어보는 , ..."
2,초조하게 시계보면서 왔다 ? 갔다 : 서성이는 김순경 . . 순찰차(경차) . 와서...
3,; 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차 ! (경차)와서 멈추고 내...
4,", 초조하게 시계 , 보면서 왔다 ? 갔다 서성이는 김순경 ? . 순찰차( . 경차..."
...,...
7334,"none , < : sep> 건강가 말하길 ""행복할 ; 일도 많다. 이까짓 ! 거 ..."
7335,"none < : sep> ? 청난가 말하길 ""종남 ? 이 맛있는것도 사주고 ? 잘 ..."
7336,"none <sep> 건강가 말 ? 하길 . ""걱정마라. 먹어도 : 같이 먹고 굶어도..."
7337,", none <sep> 종남 : 가 ? 말하길 ""아빠. 우리 저거 타."" ! 건강가..."


In [135]:
train_pairs

,scenario,label
0,"none <sep> 어영가 말하길 ""야! 전화 받아. 아무리 바빠도 내전화는 받아야...",2
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep>...",2
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,1
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,2
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,0
...,...,...
7334,"none <sep> 건강가 말하길 ""행복할 일도 많다. 이까짓거 갖고. 일단 끊어라...",0
7335,"none <sep> 청난가 말하길 ""종남이 맛있는것도 사주고 잘 챙겨줘."" 건강가 ...",0
7336,"none <sep> 건강가 말하길 ""걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다...",0
7337,"none <sep> 종남가 말하길 ""아빠. 우리 저거 타."" 건강가 말하길 ""나 못...",0


In [136]:
train_pairsxx = pd.concat([train_pairs,results_data],ignore_index=True)


train_pairsxx

,scenario,label
0,"none <sep> 어영가 말하길 ""야! 전화 받아. 아무리 바빠도 내전화는 받아야...",2
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep>...",2
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,1
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,2
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,0
...,...,...
14673,"none , < : sep> 건강가 말하길 ""행복할 ; 일도 많다. 이까짓 ! 거 ...",0
14674,"none < : sep> ? 청난가 말하길 ""종남 ? 이 맛있는것도 사주고 ? 잘 ...",0
14675,"none <sep> 건강가 말 ? 하길 . ""걱정마라. 먹어도 : 같이 먹고 굶어도...",0
14676,", none <sep> 종남 : 가 ? 말하길 ""아빠. 우리 저거 타."" ! 건강가...",0


In [137]:

train_pairsxx.to_csv(data_path+'/AEDA_aug_train_data.csv', index=False)



In [138]:
ss = pd.read_csv(data_path+'/AEDA_aug_train_data.csv', encoding='utf-8')


ss

,scenario,label
0,"none <sep> 어영가 말하길 ""야! 전화 받아. 아무리 바빠도 내전화는 받아야...",2
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep>...",2
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,1
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,2
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,0
...,...,...
14673,"none , < : sep> 건강가 말하길 ""행복할 ; 일도 많다. 이까짓 ! 거 ...",0
14674,"none < : sep> ? 청난가 말하길 ""종남 ? 이 맛있는것도 사주고 ? 잘 ...",0
14675,"none <sep> 건강가 말 ? 하길 . ""걱정마라. 먹어도 : 같이 먹고 굶어도...",0
14676,", none <sep> 종남 : 가 ? 말하길 ""아빠. 우리 저거 타."" ! 건강가...",0
